In [2]:
import numpy as np
import PIL
import PIL.Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.applications import ResNet101V2
from tensorflow.keras.applications.resnet_v2 import preprocess_input
import tensorflow.keras.backend as K
import tensorboard


In [ ]:
directory='Turkish_Food_404'

In [ ]:
batch_size= 128
img_height = 299
img_width = 299

In [ ]:
train_ds=tf.keras.preprocessing.image_dataset_from_directory(
    directory,
    labels="inferred",
    label_mode='categorical',
    batch_size=batch_size,
    image_size=(img_height,img_width),
    validation_split=0.2,
    seed=0,
    subset="training",
    interpolation="bilinear",
)

In [ ]:
val_ds=tf.keras.preprocessing.image_dataset_from_directory(
    directory,
    labels="inferred",
    label_mode='categorical',
    batch_size=batch_size,
    image_size=(img_height,img_width),
    validation_split=0.2,
    seed=0,
    subset="validation",
    interpolation="bilinear"
)

In [ ]:
import matplotlib.pyplot as plt

class_names=train_ds.class_names

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[np.argmax(labels, axis=-1)[i]])
        plt.axis("off")

In [ ]:
data_augmentation = keras.Sequential(
    [
        keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
        keras.layers.experimental.preprocessing.RandomRotation(0.3),
        keras.layers.experimental.preprocessing.RandomZoom(0.3),
    ]
    
)

In [ ]:
K.clear_session()

inputs = layers.Input(shape=(img_height,img_width, 3))
x = data_augmentation(inputs)
x = tf.cast(x, tf.float32)
#preprocess data for ResNetV2
x = preprocess_input(x)

#getting pretrained model
base_model = ResNet101V2(weights="imagenet" ,include_top=False, input_tensor=inputs)
b_model = base_model(x)


x = layers.GlobalAveragePooling2D()(b_model)
x = layers.BatchNormalization()(x)
outputs = layers.Dense(93, activation='softmax',use_bias=True , kernel_initializer="random_normal",bias_initializer="zeros")(x)
model= keras.Model(inputs=inputs,outputs=outputs)

for layer in base_model.layers:
    layer.trainable = False


In [ ]:
model.summary()

In [ ]:
opt = keras.optimizers.RMSprop(learning_rate=0.000001,momentum=0.9)

model.compile(
    optimizer=opt,  # Optimizer
    # Loss function to minimize
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    # List of metrics to monitor
    metrics=["accuracy","top_k_categorical_accuracy"],
)

In [ ]:
epochs=30


history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
)

In [3]:
#if you want to use trained model

model= keras.models.load_model("trained_models/Resnet101_best.h5")

In [ ]:
model.evaluate(val_ds)

In [ ]:
top_vals = 3 #it means code will give you top 3 prediction if you change it to 5 code will show you the top 5 predictions


image=tf.keras.preprocessing.image.load_img("./Test_Pictures/manti.jpg")
input_arr = keras.preprocessing.image.img_to_array(image)
input_arr = np.array([input_arr])
resize=tf.keras.layers.experimental.preprocessing.Resizing(299, 299, interpolation="bilinear")(input_arr)


predic = model.predict(resize,batch_size=1)

class_names=val_ds.class_names


results=np.flip(np.argsort(predic, axis=1)[:,-top_vals:][0])

for i in results:
    print(class_names[i])


In [ ]:
#del model

In [ ]:
#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

In [ ]:
#tf.config.experimental.list_physical_devices('GPU')

In [ ]:
#tf.test.is_gpu_available()